In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.pyplot import MultipleLocator

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
# Read X data set into a pandas DataFrame named X
X = pd.read_csv('./Periodic/X.csv')
# Drop the time stamp column
Y = pd.read_csv('./Periodic/Y.csv')

X

,TimeStamp,0_all_..usr,0_all_..sys,0_all_..iowait,0_all_..soft,0_all_..idle,0_cpu0_.usr,0_cpu0_.sys,0_cpu0_.iowait,0_cpu0_.soft,...,36_RxBytes.1,36_TxBytes.1,40_RxPacktes.1,40_TxPacktes.1,40_RxBytes.1,40_TxBytes.1,41_RxPacktes.1,41_TxPacktes.1,41_RxBytes.1,41_TxBytes.1
0,2016-11-20 07:13:51,2.23,0.55,0.42,0.08,96.72,2.02,0.00,1.01,0.0,...,1669523,2185488,296,365,174976,193695,367,296,121659,293712
1,2016-11-20 07:13:58,3.48,0.76,1.51,0.13,94.13,3.09,1.03,3.09,0.0,...,3082214,4331494,3483,4223,1705307,2557549,4167,3487,1603670,2822663
2,2016-11-20 07:14:01,5.40,0.79,5.36,0.17,88.28,5.05,0.00,8.08,0.0,...,2743505,3692388,2756,3491,1464216,1996217,3500,2755,1272053,2442247
3,2016-11-20 07:14:02,3.35,0.75,2.76,0.17,92.97,3.92,0.98,0.98,0.0,...,2873772,3580065,2957,3579,1619376,2006284,3565,2927,1286837,2663940
4,2016-11-20 07:14:03,3.19,0.71,5.71,0.21,90.18,1.02,0.00,3.06,0.0,...,2819232,3679884,3030,3599,1533629,2169831,3625,3051,1373232,2557414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28957,2016-11-20 19:42:06,0.13,0.21,0.04,0.00,99.62,0.00,0.00,1.01,0.0,...,177084,184781,291,389,179191,196811,389,291,122963,296479
28958,2016-11-20 19:42:12,0.04,0.13,0.04,0.00,99.79,0.00,0.00,1.00,0.0,...,184501,150767,296,369,196312,175558,375,305,110806,332621
28959,2016-11-20 19:42:19,0.04,0.08,0.00,0.00,99.87,0.00,0.00,0.00,0.0,...,168011,265356,321,443,173265,246279,444,321,156563,289105
28960,2016-11-20 19:42:36,0.04,0.08,0.00,0.00,99.87,0.00,0.00,1.00,0.0,...,174003,209297,310,429,172716,236146,430,310,147908,284212


In [4]:
Y_D=Y.drop("TimeStamp",1)
X_D=X.drop("TimeStamp",1)

XY = pd.concat([X_D,Y['ReadsAvg'],Y['WritesAvg']], axis=1)
XY_= pd.concat([X,Y['ReadsAvg'],Y['WritesAvg']], axis=1)
XY_sample=XY_.sample(n=int(X.shape[0]/2))

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(XY_sample, test_size = 0.3,random_state = 2)

# Sort the train and test sets after index (which became unsorted through sampling)
train = train.sort_index(axis=0)
test = test.sort_index(axis=0)
train.to_csv("./Periodic/train.csv")
test.to_csv("./Periodic/test.csv")


In [29]:
train = pd.read_csv('./Periodic/train.csv')
# Drop the time stamp column
test = pd.read_csv('./Periodic/test.csv')
train= train.loc[:, (train != 0).any(axis=0)]
#test=???

In [30]:
train

,Unnamed: 0,TimeStamp,0_all_..usr,0_all_..sys,0_all_..iowait,0_all_..soft,0_all_..idle,0_cpu0_.usr,0_cpu0_.sys,0_cpu0_.iowait,...,40_RxPacktes.1,40_TxPacktes.1,40_RxBytes.1,40_TxBytes.1,41_RxPacktes.1,41_TxPacktes.1,41_RxBytes.1,41_TxBytes.1,ReadsAvg,WritesAvg
0,0,2016-11-20 07:13:51,2.23,0.55,0.42,0.08,96.72,2.02,0.00,1.01,...,296,365,174976,193695,367,296,121659,293712,73.8254,175.5445
1,3,2016-11-20 07:14:02,3.35,0.75,2.76,0.17,92.97,3.92,0.98,0.98,...,2957,3579,1619376,2006284,3565,2927,1286837,2663940,56.9347,108.4472
2,6,2016-11-20 07:14:05,3.89,0.96,3.43,0.13,91.58,0.00,0.00,4.12,...,2719,3464,1521516,2071803,3463,2740,1311750,2596759,58.3944,107.6670
3,8,2016-11-20 07:14:10,1.46,0.63,1.80,0.13,95.99,1.02,0.00,2.04,...,5081,6497,1949522,2803562,6486,5090,1819787,2839599,59.4581,120.0390
4,9,2016-11-20 07:14:11,0.75,0.25,1.05,0.04,97.91,1.00,0.00,0.00,...,3951,4591,1854630,2411463,4590,3951,1545479,3048829,55.9895,114.3459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10131,28955,2016-11-20 19:42:03,0.08,0.08,0.00,0.00,99.83,0.00,0.00,0.00,...,934,1361,489817,541623,1369,934,440170,605657,53.6099,105.9537
10132,28956,2016-11-20 19:42:05,0.17,0.04,0.00,0.00,99.79,0.00,0.00,0.00,...,274,345,187430,154210,346,274,97798,310510,55.5818,109.0576
10133,28957,2016-11-20 19:42:06,0.13,0.21,0.04,0.00,99.62,0.00,0.00,1.01,...,291,389,179191,196811,389,291,122963,296479,55.6183,107.0251
10134,28958,2016-11-20 19:42:12,0.04,0.13,0.04,0.00,99.79,0.00,0.00,1.00,...,296,369,196312,175558,375,305,110806,332621,52.6730,104.5830


In [31]:
train_ = train.drop("TimeStamp",1)
test_= train.drop("TimeStamp",1)

In [32]:
train.columns

Index(['Unnamed: 0', 'TimeStamp', '0_all_..usr', '0_all_..sys',
       '0_all_..iowait', '0_all_..soft', '0_all_..idle', '0_cpu0_.usr',
       '0_cpu0_.sys', '0_cpu0_.iowait',
       ...
       '40_RxPacktes.1', '40_TxPacktes.1', '40_RxBytes.1', '40_TxBytes.1',
       '41_RxPacktes.1', '41_TxPacktes.1', '41_RxBytes.1', '41_TxBytes.1',
       'ReadsAvg', 'WritesAvg'],
      dtype='object', length=1746)

In [33]:
# Extract X,Y components from test and train sets
X_train = train.drop("TimeStamp",1)
X_train = X_train.drop('Unnamed: 0',1)
X_train = X_train.drop("ReadsAvg",1)
X_train = X_train.drop("WritesAvg",1)
X_test = test.drop("TimeStamp",1)
X_test = X_test.drop('Unnamed: 0',1)
X_test = X_test.drop("ReadsAvg",1)
X_test = X_test.drop("WritesAvg",1)
Y_train_r = train["ReadsAvg"]
#Y_train_w = train["WritesAvg"]
Y_test_r = test["ReadsAvg"]
#Y_test_w = test["WritesAvg"]

In [14]:
X_train

,0_all_..usr,0_all_..sys,0_all_..iowait,0_all_..soft,0_all_..idle,0_cpu0_.usr,0_cpu0_.sys,0_cpu0_.iowait,0_cpu0_.soft,0_cpu0_.idle,...,36_RxBytes.1,36_TxBytes.1,40_RxPacktes.1,40_TxPacktes.1,40_RxBytes.1,40_TxBytes.1,41_RxPacktes.1,41_TxPacktes.1,41_RxBytes.1,41_TxBytes.1
0,2.23,0.55,0.42,0.08,96.72,2.02,0.00,1.01,0.0,96.97,...,1669523,2185488,296,365,174976,193695,367,296,121659,293712
1,3.35,0.75,2.76,0.17,92.97,3.92,0.98,0.98,0.0,94.12,...,2873772,3580065,2957,3579,1619376,2006284,3565,2927,1286837,2663940
2,3.89,0.96,3.43,0.13,91.58,0.00,0.00,4.12,0.0,95.88,...,2754331,3749209,2719,3464,1521516,2071803,3463,2740,1311750,2596759
3,1.46,0.63,1.80,0.13,95.99,1.02,0.00,2.04,0.0,96.94,...,6134161,7405346,5081,6497,1949522,2803562,6486,5090,1819787,2839599
4,0.75,0.25,1.05,0.04,97.91,1.00,0.00,0.00,0.0,99.00,...,3437203,4710580,3951,4591,1854630,2411463,4590,3951,1545479,3048829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10131,0.08,0.08,0.00,0.00,99.83,0.00,0.00,0.00,0.0,100.00,...,801878,926605,934,1361,489817,541623,1369,934,440170,605657
10132,0.17,0.04,0.00,0.00,99.79,0.00,0.00,0.00,0.0,100.00,...,181853,164944,274,345,187430,154210,346,274,97798,310510
10133,0.13,0.21,0.04,0.00,99.62,0.00,0.00,1.01,0.0,98.99,...,177084,184781,291,389,179191,196811,389,291,122963,296479
10134,0.04,0.13,0.04,0.00,99.79,0.00,0.00,1.00,0.0,99.00,...,184501,150767,296,369,196312,175558,375,305,110806,332621


In [15]:
### Feature selection

In [16]:
'''
start_time = time.time()
corr=train_.corr()
corr['sq_r'] = corr.ReadsAvg**2
corr_r = corr.sort_values('sq_r',ascending=False)
end_time = time.time()
print(end_time-start_time)
'''

"\nstart_time = time.time()\ncorr=train_.corr()\ncorr['sq_r'] = corr.ReadsAvg**2\ncorr_r = corr.sort_values('sq_r',ascending=False)\nend_time = time.time()\nprint(end_time-start_time)\n"

In [17]:
#better method
pccs=np.ones((X_train.shape[1],1))
start_time = time.time()
for k in range(0,X_train.shape[1]):
    pccs[k]=(np.corrcoef(X_train.iloc[:,k], Y_train_r)[0,1])
end_time = time.time()
print(end_time-start_time)

0.6145105361938477


In [18]:
pccs_df=pd.DataFrame(pccs,columns=['score'])
pccs_df.index=X_train.columns[0:X_train.shape[1]]
pccs_df=pccs_df.sort_values(by='score',ascending=False)
pccs_df.to_csv("./Periodic/read_p_c.csv")

In [19]:
#corr_r['sq_r'].to_csv("./Flash/read_p_c.csv")
#corr_r['sq_r'].head(64).to_csv("read_p_c.csv")

In [34]:
>>> from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

Y_train_r_int=Y_train_r.astype(int)
start_time = time.time()
clf=ExtraTreesClassifier(n_estimators=100)
clf.fit(X_train, Y_train_r_int)
feat_importances = pd.Series(clf.feature_importances_, index=X_train.columns)
end_time = time.time()
print(end_time-start_time)

12.898924350738525


In [35]:
#feat_importances.nlargest(64).to_csv("read_t_b.csv")
feat_importances.nlargest(feat_importances.size).to_csv("./Periodic/read_t_b.csv")

In [22]:
from numpy import dot
from numpy.linalg import norm

exp=X_train
exp= exp.loc[:, (exp != 0).any(axis=0)]
exp.loc[-1] = range(0,exp.shape[1])

#arr=exp[flist16]
arr=exp
arr_d_x=arr.shape[0]
arr_d_y=arr.shape[1]

In [23]:
start_time = time.time()
dotab=np.zeros((arr_d_y,arr.shape[1]))
for i in range(0,arr_d_y):
    a=exp.iloc[0:arr_d_x-1,i]
    for j in range(0,i+1):
        b=arr.iloc[0:arr_d_x-1,j]
        dotab[i,j]=dot(a, b)
end_time = time.time()
print(end_time-start_time)

427.93058347702026


In [24]:
start_time = time.time()
for i in range(0,arr_d_y):
    for j in range(i+1,arr_d_y):
        dotab[i,j]=dotab[j, i]
narr=np.zeros(arr_d_y)
for i in range(0,arr_d_y):
    a=arr.iloc[0:arr_d_x-1,i]
    na=norm(a)
    narr[i]=na
end_time = time.time()
print(end_time-start_time)
narr

2.0600790977478027


array([5.93283735e+01, 3.58658863e+01, 2.63618076e+01, ...,
       3.76441191e+05, 1.82613621e+08, 3.45235221e+08])

In [25]:
start_time = time.time()
for i in range(0,arr_d_y):
    mean=arr.iloc[:,i].mean()
    r=0.0
    for j in range(0,arr_d_x-1):
        r=r+np.abs(arr.iloc[j,i]-mean)
    s=0.0
    for j in range(0,arr_d_y):
        na=narr[i]
        nb=narr[j]
        if((na>0)and(nb>0)):
            s=s+np.abs(dotab[i,j]/(na*nb))
    if(s!=0):
        score=r/s
    else:
        score=math.inf
    arr.iloc[-1,i]=score
end_time = time.time()
print(end_time-start_time)
arr

684.0682096481323


,0_all_..usr,0_all_..sys,0_all_..iowait,0_all_..soft,0_all_..idle,0_cpu0_.usr,0_cpu0_.sys,0_cpu0_.iowait,0_cpu0_.soft,0_cpu0_.idle,...,36_RxBytes.1,36_TxBytes.1,40_RxPacktes.1,40_TxPacktes.1,40_RxBytes.1,40_TxBytes.1,41_RxPacktes.1,41_TxPacktes.1,41_RxBytes.1,41_TxBytes.1
0,2.230000,0.550000,0.420000,0.080000,96.720000,2.020000,0.000000,1.010000e+00,0.000000,96.97000,...,1.669523e+06,2.185488e+06,296.00000,365.000000,1.749760e+05,1.936950e+05,367.000000,296.000000,1.216590e+05,2.937120e+05
1,3.350000,0.750000,2.760000,0.170000,92.970000,3.920000,0.980000,9.800000e-01,0.000000,94.12000,...,2.873772e+06,3.580065e+06,2957.00000,3579.000000,1.619376e+06,2.006284e+06,3565.000000,2927.000000,1.286837e+06,2.663940e+06
2,3.890000,0.960000,3.430000,0.130000,91.580000,0.000000,0.000000,4.120000e+00,0.000000,95.88000,...,2.754331e+06,3.749209e+06,2719.00000,3464.000000,1.521516e+06,2.071803e+06,3463.000000,2740.000000,1.311750e+06,2.596759e+06
3,1.460000,0.630000,1.800000,0.130000,95.990000,1.020000,0.000000,2.040000e+00,0.000000,96.94000,...,6.134161e+06,7.405346e+06,5081.00000,6497.000000,1.949522e+06,2.803562e+06,6486.000000,5090.000000,1.819787e+06,2.839599e+06
4,0.750000,0.250000,1.050000,0.040000,97.910000,1.000000,0.000000,0.000000e+00,0.000000,99.00000,...,3.437203e+06,4.710580e+06,3951.00000,4591.000000,1.854630e+06,2.411463e+06,4590.000000,3951.000000,1.545479e+06,3.048829e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10132,0.170000,0.040000,0.000000,0.000000,99.790000,0.000000,0.000000,0.000000e+00,0.000000,100.00000,...,1.818530e+05,1.649440e+05,274.00000,345.000000,1.874300e+05,1.542100e+05,346.000000,274.000000,9.779800e+04,3.105100e+05
10133,0.130000,0.210000,0.040000,0.000000,99.620000,0.000000,0.000000,1.010000e+00,0.000000,98.99000,...,1.770840e+05,1.847810e+05,291.00000,389.000000,1.791910e+05,1.968110e+05,389.000000,291.000000,1.229630e+05,2.964790e+05
10134,0.040000,0.130000,0.040000,0.000000,99.790000,0.000000,0.000000,1.000000e+00,0.000000,99.00000,...,1.845010e+05,1.507670e+05,296.00000,369.000000,1.963120e+05,1.755580e+05,375.000000,305.000000,1.108060e+05,3.326210e+05
10135,0.040000,0.080000,0.000000,0.000000,99.870000,0.000000,0.000000,1.000000e+00,0.000000,99.00000,...,1.740030e+05,2.092970e+05,310.00000,429.000000,1.727160e+05,2.361460e+05,430.000000,310.000000,1.479080e+05,2.842120e+05


In [26]:
score=arr.iloc[-1,:]
score=score.sort_values(ascending=False)
score.to_csv("./Periodic/read_arr.csv")
#score.head(64).to_csv("read_arr.csv")

In [27]:
### PCA